# GAN

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import fastai
import torch
from fastai.vision.all import *
from fastai.vision.gan import *

In [3]:
print(fastai.__version__) # version check

2.7.15


### Data

In [4]:
embedding = torch.load('embedding.pkl')

In [7]:
class Txt2ImgYTransform(Transform):
    def __init__(self, embedding=embedding, **kwargs):
        self.embedding = embedding
    
    def encodes(self, fn):
        key = fn.stem + '.txt'
        img = PILImage.create(fn)
        return (TensorImage(self.embedding[key]).cuda(), img)
    
    def decodes(self, x):
        return PILImage(x[1].float().clamp(min=0, max=1))
        
    def show(self, xs, ys=None, zs=None, imgsize=4, figsize=None, **kwargs):
        raise NotImplementedError

In [8]:
class Txt2ImgXTransform(Transform):
    def __init__(self, embedding=embedding, **kwargs):
        self.embedding = embedding
    
    def encodes(self, fn):
        key = fn.stem + '.txt'
        img = self.get_wrong_image(fn)
        return (TensorImage(self.embedding[key]).cuda(), img)
    
    def get_wrong_image(self, fn):
        cat = '_'.join(fn.name.split('_')[:-3])
        items = fn.ls()  # List of all items in the directory
        idx = np.random.randint(len(items))
        
        while items[idx].name.startswith(cat):
            idx = np.random.randint(len(items))
        
        return PILImage.create(items[idx])

    def decodes(self, x):
        return PILImage(x[1].float().clamp(min=0, max=1))
        
    def show(self, xs, ys=None, zs=None, imgsize=4, figsize=None, **kwargs):
        raise NotImplementedError

In [ ]:
data = (Txt2ImgXItemList
            .from_folder('data/images/')
            .split_none()            
            .label_from_func(noop)
            .databunch(bs=128, num_workers=0)) # num_workers: for Windows
            # TODO: normalize?